### Effective Energy Shift Algorithm for Electric Energy Storage Analysis

This notebook is intended as an interactive playground to foster the understanding of the Effective Energy Shift (EfES) algorithm.

#### Import modules and define functions for visualization

In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interactive_output, fixed, interact_manual
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.collections import LineCollection
import matplotlib as mpl

import numpy as np

import effective_energy_shift as efes
import efes_dataclasses as efes_dc
import math_energy_systems as mes

def plot_input(results: efes_dc.Results): 
    fig, axs = plt.subplots(3,1, sharex=True, figsize=(15,4))
    time=np.arange(0, results.analysis_results.data_input.delta_time_step*len(results.analysis_results.data_input.power_residual_generation), results.analysis_results.data_input.delta_time_step)
    x = np.array([*time, time[-1]+1])
    
    y = np.array([*results.analysis_results.data_input.power_generation, results.analysis_results.data_input.power_generation[-1]])
    axs[0].step(x=x, y=y, where='post', color='green', label='$\mathit{P}_{\mathrm{gen}}$')
    axs[0].fill_between(x=x, y1=y, step='post', facecolor='green', alpha=0.2, label='$\mathit{E}_{\mathrm{gen}}=$'+efes.pretty_print(results.analysis_results.energy_generation, 'Wh', decimals=0))
    y = np.array([*results.analysis_results.data_input.power_used_generation, results.analysis_results.data_input.power_used_generation[-1]])
    axs[0].fill_between(x=x, y1=y, step='post', facecolor='white', hatch='//////', alpha=0.2, label='$\mathit{E}_{\mathrm{ugen}}=$'+efes.pretty_print(results.analysis_results.energy_used_generation, 'Wh', decimals=0))
    lgd = axs[0].legend(bbox_to_anchor=(1.0, 1.0))
    handles, labels = axs[0].get_legend_handles_labels()
    handles.append(Patch(facecolor='w', edgecolor='w'))
    labels.append('$\psi_{\mathrm{sc}}=$'+f'{results.analysis_results.self_consumption_initial:.2f}')
    lgd._legend_box = None
    lgd._init_legend_box(handles, labels)
    lgd._set_loc(lgd._loc)
    lgd.set_title(lgd.get_title().get_text())
    
    y = np.array([*results.analysis_results.data_input.power_demand, results.analysis_results.data_input.power_demand[-1]])
    axs[1].step(x=x, y=y, where='post', color='red', label='$\mathit{P}_{\mathrm{dem}}$')
    axs[1].fill_between(x=x, y1=y, step='post', facecolor='red', alpha=0.2, label='$\mathit{E}_{\mathrm{dem}}=$'+efes.pretty_print(results.analysis_results.energy_demand, 'Wh', decimals=0))
    y = np.array([*results.analysis_results.data_input.power_covered_demand, results.analysis_results.data_input.power_covered_demand[-1]])
    axs[1].fill_between(x=x, y1=y, step='post', facecolor='white', hatch='//////', alpha=0.2, label='$\mathit{E}_{\mathrm{cdem}}=$'+efes.pretty_print(results.analysis_results.energy_covered_demand, 'Wh', decimals=0))
    lgd = axs[1].legend(bbox_to_anchor=(1.0, 1.0))
    handles, labels = axs[1].get_legend_handles_labels()
    handles.append(Patch(facecolor='w', edgecolor='w'))
    labels.append('$\psi_{\mathrm{ss}}=$'+f'{results.analysis_results.self_sufficiency_initial:.2f}')
    lgd._legend_box = None
    lgd._init_legend_box(handles, labels)
    lgd._set_loc(lgd._loc)
    lgd.set_title(lgd.get_title().get_text())
    
    y = np.array([*results.analysis_results.data_input.power_residual_generation, results.analysis_results.data_input.power_residual_generation[-1]])
    axs[2].step(x=x, y=y, where='post', color='black', label='$\mathit{P}_{\mathrm{rgen}}$')
    power_excess_initial = np.clip(y,a_min=0, a_max=np.inf)
    axs[2].fill_between(x=x, y1=power_excess_initial, step='post', color='green', alpha=0.2, label='$\mathit{E}_{\mathrm{exs}}=$'+efes.pretty_print(power_excess_initial[:-1].sum()*results.analysis_results.data_input.delta_time_step, 'Wh', decimals=0))
    power_deficit_initial = np.clip(y,a_min=-np.inf, a_max=0)
    axs[2].fill_between(x=x, y1=power_deficit_initial, step='post', color='red', alpha=0.2, label='$\mathit{E}_{\mathrm{def}}=$'+efes.pretty_print(-power_deficit_initial[:-1].sum()*results.analysis_results.data_input.delta_time_step, 'Wh', decimals=0))
    lgd = axs[2].legend(bbox_to_anchor=(1.0, 1.0))
    handles, labels = axs[2].get_legend_handles_labels()
    handles.append(Patch(facecolor='w', edgecolor='w'))
    labels.append('$\psi_{\mathrm{sc,max}}=$'+f'{results.analysis_results.self_consumption_max:.2f}')
    handles.append(Patch(facecolor='w', edgecolor='w'))
    labels.append('$\psi_{\mathrm{ss,max}}=$'+f'{results.analysis_results.self_sufficiency_max:.2f}')
    lgd._legend_box = None
    lgd._init_legend_box(handles, labels)
    lgd._set_loc(lgd._loc)
    lgd.set_title(lgd.get_title().get_text())
    
    axs[0].set(ylabel='Power [W]')
    axs[1].set(ylabel='Power [W]')
    axs[2].set(ylabel='Power [W]', xlabel='Time [h]', xlim=(x.min(), x.max()))
    axs[0].grid()
    axs[1].grid()
    axs[2].grid()
    fig.tight_layout()
    plt.show()

def plot_results(results):
    fig, axs = plt.subplots(2,1, sharex=True, figsize=(15,4))    
        
    x = results.analysis_results.capacity
    x = np.append(x, x[-1]*1.5)
    y = [*results.analysis_results.energy_additional,results.analysis_results.energy_additional[-1]]
    ylim = [0, y[-1]*1.2]
    
    axs[0].plot(x, y)    
    axs[0].plot([x[0],x[-1]], [results.analysis_results.energy_additional[-1], results.analysis_results.energy_additional[-1]], linestyle='--',color='black', linewidth=2)
    axs[0].add_artist(mpl.text.Text(x=0.02*x[-1], y=1.01*results.analysis_results.energy_additional[-1], text=f'Max: {efes.pretty_print(results.analysis_results.energy_additional[-1],"Wh")}', clip_on=False, horizontalalignment='left',verticalalignment='bottom'))
    
    def create_axes(axs, axes_x, tick_lim, tick_width, label_offset, label_lim_offset, axes_title, axes_func):    
        line = Line2D([x[-1]*axes_x,x[-1]*axes_x], [0, y[-1]], lw=1., color='black')
        line.set_clip_on(False)
        axs.add_artist(line)
    
        ticks = np.linspace(tick_lim[0], tick_lim[-1], 100)    
        decimals = 5
        while len(ticks) > 8:
            ticks = np.unique(np.round(ticks, decimals))        
            decimals = decimals - 1        
        
        ticks = ticks[(ticks>=1.01*tick_lim[0]) & (ticks<=0.99*tick_lim[-1])]
        
        ticks_loc = axes_func(ticks)
            
        axs.add_artist(LineCollection(segments = [((x[-1]*(axes_x-tick_width), loc), (x[-1]*(axes_x+tick_width), loc)) for (loc, label) in zip(ticks_loc, ticks)], linewidth=1, clip_on=False, color='black'))
        axs.add_artist(LineCollection(segments = [((x[-1]*(axes_x-2*tick_width), loc), (x[-1]*(axes_x+2*tick_width), loc)) for (loc, label) in zip([0, y[-1]], tick_lim)], linewidth=2, clip_on=False, color='black'))
        
        axs.add_artist(mpl.text.Text(x=x[-1]*(axes_x+tick_width+label_lim_offset), y=0, text=f'{np.round(tick_lim[0], decimals+2)}', clip_on=False, verticalalignment='top'))
        axs.add_artist(mpl.text.Text(x=x[-1]*(axes_x+tick_width+label_lim_offset), y=y[-1], text=f'{np.round(tick_lim[-1], decimals+2)}', clip_on=False, verticalalignment='bottom'))
        
        for loc, label in zip(ticks_loc, ticks):           
            axs.add_artist(mpl.text.Text(x=x[-1]*(axes_x+tick_width+label_offset), y=loc, text=f'{label}', clip_on=False, verticalalignment='center'))
    
        axs.add_artist(mpl.text.Text(x=x[-1]*axes_x, y=1.1*y[-1], text=axes_title, clip_on=False, horizontalalignment='left',verticalalignment='bottom'))

    def axes_func(ticks):
        return mes.calculate_additional_energy_from_self_sufficiency(self_sufficiency=ticks, energy_demand=results.analysis_results.energy_demand, self_sufficiency_initial=results.analysis_results.self_sufficiency[0])
        
    create_axes(axs[0], 
                axes_x = 1.01,
                tick_width = 0.002,
                label_offset = 0.001,
                label_lim_offset = 0.005,
                axes_title = r'$\psi_{\mathrm{ss}}$',
                tick_lim = (results.analysis_results.self_sufficiency[0], results.analysis_results.self_sufficiency[-1]), 
                axes_func=axes_func
                )

    def axes_func(ticks):
        return mes.calculate_additional_energy_from_self_consumption(self_consumption=ticks, energy_generation=results.analysis_results.energy_generation, self_consumption_initial=results.analysis_results.self_consumption[0], efficiency_discharging=results.analysis_results.data_input.efficiency_discharging, efficiency_charging=results.analysis_results.data_input.efficiency_charging)
        
    create_axes(axs[0], 
                axes_x = 1.05,
                tick_width = 0.002,
                label_offset = 0.001,
                label_lim_offset = 0.005,
                axes_title = r'$\psi_{\mathrm{sc}}$',
                tick_lim = (results.analysis_results.self_consumption[0], results.analysis_results.self_consumption[-1]), 
                axes_func=axes_func
                )

    axs[1].plot(results.query_results[0].capacity, results.query_results[0].gain, label='gain $\mathit{G}$', linestyle='-.', linewidth=2)
    axs[1].plot(results.query_results[0].capacity, results.query_results[0].gain/results.analysis_results.data_input.efficiency_discharging, label='effectiveness $\mu$')
    axs[1].step(x, [*results.analysis_results.effectiveness,0], where='post', label='local effectiveness $\mathit{m}$')
    axs[1].legend()
    
    axs[0].grid()
    axs[1].grid()    
    axs[0].set(ylabel='$\mathit{E}^{+}$ [Wh]', ylim=ylim)
    axs[1].set(ylabel='$\mathit{G}$, $\mu$ and $\mathit{m}$ [1]', xlabel='$C$ [Wh]', xlim=(x.min(), x.max()))
    fig.tight_layout()
    plt.show()

#### Interactive example from the publication

In [5]:
power_generation = np.array([2,3,2,4,3,1,0,0,2,5,6,2,1,0,1,2,3,2,0,0,4,4,4,2])
power_demand = np.array([1,4,1,2,1,2,4,5,0,1,3,1,2,2,1,1,2,3,4,5,2,1,5,1])

energy_generation_original = power_generation.sum()
power_generation_normalized = power_generation/energy_generation_original

energy_demand_original = power_demand.sum()
power_demand_normalized = power_demand/energy_demand_original

def update(energy_demand, energy_generation, efficiency_charging, efficiency_discharging, efficiency_direct_usage):
    
    power_generation = energy_generation*power_generation_normalized
    power_demand = energy_demand*power_demand_normalized
    
    delta_time_step = 1.
    
    results = efes.perform_energy_storage_dimensioning(
        power_generation=power_generation,
        power_demand=power_demand,
        delta_time_step=delta_time_step, 
        power_max_charging=np.inf, 
        efficiency_direct_usage=efficiency_direct_usage,
        efficiency_discharging=efficiency_discharging, 
        efficiency_charging=efficiency_charging
    )
    results.query_results=[efes.run_dimensioning_query_for_target_capacity(results.analysis_results, capacity_target=np.linspace(0.001*results.analysis_results.capacity[-1],results.analysis_results.capacity[-1]*1.5,500))]
    plot_input(results)
    #plot_initial_phases(results)
    plot_results(results)

style = dict(description_width='initial', handle_color='lightblue')

widget_energy_demand=widgets.FloatText(
    value=energy_demand_original,
    description='energy demand:',
    style = style,
    disabled=False
)
widget_energy_generation=widgets.FloatText(
    value=energy_generation_original,
    description='energy generation:',
    style = style,
    disabled=False
)
widget_efficiency_charging=widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=1.0,
    step=0.01,
    description='efficiency charging:',
    style = style,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
widget_efficiency_discharging=widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=1.0,
    step=0.01,
    description='efficiency discharging:',
    style = style,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
widget_efficiency_direct_usage=widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=1.0,
    step=0.01,
    description='efficiency direct usage:',
    style = style,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
ui = widgets.VBox([widgets.HBox([widget_energy_demand, widget_energy_generation]), widgets.HBox([widget_efficiency_direct_usage, widget_efficiency_charging, widget_efficiency_discharging])])

out = interactive_output(update, dict(energy_demand=widget_energy_demand,
                             energy_generation=widget_energy_generation,
                             efficiency_charging=widget_efficiency_charging,
                             efficiency_discharging=widget_efficiency_discharging,
                             efficiency_direct_usage=widget_efficiency_direct_usage
                        ))
display(ui,out)

Output()